In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [99]:
data = pd.read_csv("battles_data.csv")

In [101]:
data.loc[201]

id                                                     vJVgFxeyJmg
title            "Breakthrough 2" // Runde 2 // D-Kay vs Neilz ...
upload_date                                             20201206.0
view_count                                                   33624
like_count                                                     819
comment_count                                                169.0
url                    https://www.youtube.com/watch?v=vJVgFxeyJmg
status                                                      active
Name: 201, dtype: object

In [103]:
data. info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1120 entries, 0 to 1119
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1120 non-null   object 
 1   title          1115 non-null   object 
 2   upload_date    1115 non-null   float64
 3   view_count     1120 non-null   int64  
 4   like_count     1120 non-null   int64  
 5   comment_count  1119 non-null   float64
 6   url            1120 non-null   object 
 7   status         1120 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 70.1+ KB


In [105]:
df = data[~data["title"].str.contains('Ankündigung|Freestyle|Teaser|Provokation|Ansage|Trailer|Breakthrough', na=False)]

In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1049 entries, 0 to 1119
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1049 non-null   object 
 1   title          1044 non-null   object 
 2   upload_date    1044 non-null   float64
 3   view_count     1049 non-null   int64  
 4   like_count     1049 non-null   int64  
 5   comment_count  1048 non-null   float64
 6   url            1049 non-null   object 
 7   status         1049 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 73.8+ KB


In [109]:
# title-Spalte splitten

# Alte Spalte umbenennen
df = df.rename(columns={'title': 'title_raw'})


# Verschiedene Pipe-Varianten durch ASCII-Pipe normalisieren
df['title_raw'] = df['title_raw'].str.replace(r'[⎪¦｜‖]', '|', regex=True)





In [111]:
# Prüfen: enthält 'vs' (case-insensitive)
df['has_vs'] = df['title_raw'].str.contains('vs', case=False, na=False)

df[['title_raw', 'has_vs']].head(10)

# Nur Zeilen behalten, die 'vs' enthalten
df = df[df['has_vs'] == True]#.reset_index(drop=True)

# Optional: die Hilfsspalte entfernen
df = df.drop(columns=['has_vs'])

In [113]:
# Splitten am ersten Vorkommen von '|' oder '//', nicht-einfangend
split_result = df['title_raw'].str.split(r'\s*(?:\||//)\s*', n=1, expand=True)

# Sicherstellen, dass immer zwei Spalten existieren
if split_result.shape[1] == 1:
    split_result[1] = ''
    
# Neue Spalten anlegen
df['title'] = split_result[0].str.strip()
df['info'] = split_result[1].str.strip()

In [115]:
# Contender trennen

# Regex: ersetze alle Varianten von vs/ vs. durch 'vs.'
df['title'] = df['title'].str.replace(r'(?i)\bvs\.{0,}(?=[^\w]|$)', 'vs.', case=False, regex=True)

# Split am ersten 'vs.'
split_result = df['title'].str.split(r'\s+vs\.\s+', n=1, expand=True)

# Neue Spalten
df['name1'] = split_result[0].str.strip()
df['name2'] = split_result[1].str.strip()

In [117]:
# Liga eintragen

df['league'] = np.where(df['info'].str.contains('DLTLLY', case=False, na=False) | 
                         df['title'].str.contains('DLTLLY', case=False, na=False), 'DLTLLY',
                  np.where(df['info'].str.contains('FOB', case=False, na=False) |
                           df['title'].str.contains('FOB', case=False, na=False), 'FOB',
                    np.where(df['info'].str.contains("Don't Flop", case=False, na=False) |
                           df['title'].str.contains("Don't Flop", case=False, na=False), "Don't Flop",
                        np.where(df['info'].str.contains('Breakthrough', case=False, na=False) |
                           df['title'].str.contains('Breakthrough', case=False, na=False), 'Breakthrough',''))))

# Zeile individuell anpassen
df.loc[724, ["league"]] = ["Don't Flop"]

# Zeilen anzeigen, bei denen 'league' leer ist
df_no_league = df[df['league'] == '']
df_no_league

,id,title_raw,upload_date,view_count,like_count,comment_count,url,status,title,info,name1,name2,league
1,SYo3zBQzwtQ,Aytee vs Karanova | 🏆 ON BEAT TITLE MATCH 2025...,20250316.0,208529,6257,755.0,https://www.youtube.com/watch?v=SYo3zBQzwtQ,active,Aytee vs. Karanova,"🏆 ON BEAT TITLE MATCH 2025 🏆| RapBattle, Berlin",Aytee,Karanova,
217,FgFwq0icwso,Kato vs Mr. Whyte // Fussball Battle // Panke ...,20200719.0,28097,730,175.0,https://www.youtube.com/watch?v=FgFwq0icwso,active,Kato vs. Mr. Whyte,Fussball Battle // Panke Garten Berlin // 2020,Kato,Mr. Whyte,
219,Tm9vJh_dk9k,Khacoby vs Notyzze // Corona Battle Package,20200628.0,27973,810,192.0,https://www.youtube.com/watch?v=Tm9vJh_dk9k,active,Khacoby vs. Notyzze,Corona Battle Package,Khacoby,Notyzze,
220,LOOhH4_OvNE,Mars B. vs BX // Corona Battle Package,20200626.0,11170,291,70.0,https://www.youtube.com/watch?v=LOOhH4_OvNE,active,Mars B. vs. BX,Corona Battle Package,Mars B.,BX,
221,bMXRfRmgPTA,Craze vs Burst // English 1 Rounder // Corona ...,20200623.0,6276,91,35.0,https://www.youtube.com/watch?v=bMXRfRmgPTA,active,Craze vs. Burst,English 1 Rounder // Corona Battle Package,Craze,Burst,
226,vb_w46X0rGE,Proton vs Craze // On Beat + Accapella // Coro...,20200607.0,21101,620,154.0,https://www.youtube.com/watch?v=vb_w46X0rGE,active,Proton vs. Craze,On Beat + Accapella // Corona Package,Proton,Craze,


In [119]:
#alles, was keinen Eintrag in "league" hat, bekommt DLTLLY (geprüft!)

df.loc[df['league'] == '', 'league'] = "DLTLLY"

In [121]:
# Ansetzungsform bereinigen, Klammern entfernen

df["name1"] = df["name1"].str.replace(r"\(.*\)", "", regex=True).str.strip()
df["name2"] = df["name2"].str.replace(r"\(.*\)", "", regex=True).str.strip()

In [123]:
# Namen in neue Spalte kopieren für Ansetzungsform > norm_name

df["norm_name #1"] = df["name1"]
df["norm_name #2"] = df["name2"]

In [125]:
# Ansetzungsform bereinigen, Groß-Kleinschreibung

df["norm_name #1"] = df["norm_name #1"].str.title()
df["norm_name #2"] = df["norm_name #2"].str.title()
df["info"] = df["info"].str.title()

In [127]:
# Aliase auflisten für norm_name-Spalten

aliases = {
    "Triplebeat": "Triple",
    "Triple Beat": "Triple",
    "TRIPLE": "Triple",  
    
    ".Tecey": "Tecey",
    
    "Alice": "Atropatrap",
    "Atzenkalle -": "Atzenkalle",

    "Bahamakarma": "Karma",
    "B-Dad": "Bdad",
    "Andres Kaliba": "Bilal",
    "Bong Teggy - Rematch": "Bong Teggy",
    "Bronko Banano Aka Street": "Bronko Banano",
    "Street": "Bronko Banano",
    
    "Cashus K.": "Cashus K",
    "Capco Jonnes": "Capco Jones",
    "Capcojones": "Capco Jones",
    "Cemo": "Cemo Dafuq",
    "Change": "Change19",
    "Cleptomatic": "Clep",
    "Clepto": "Clep",
    "Craze' Parts": "Craze",

    "D.E.Z.": "D.E.Z",
    "Der Fischer -": "Der Fischer",
    "Desperados": "Despo",

    "Ein Letztes Mal: Yarambo": "Yarambo",

    "Falk": "Falki",
    "Fatcap -": "Fatcap",
    "Fate One": "Fate",
    "Fate1": "Fate",
    "Four Seven": "Fourseven",
    "Fraxone": "Frax One",

    "Gier -": "Gier",

    "Hansen - Rematch": "Hansen",
    "Herrn Sören": "Herr Sören",

    "Jack Dragon - Rematch!": "Jack Dragon",
    "Jean Gough Frais": "Jean-Gough Frais",
    "Jizi": "Ji-Zi",
    "Juse Ju -": "Juse Ju",

    "Khacoby": "Karkobi",
    "King Lil&#39;C'": "King Lil'C",
    "Kol&#39;ja": "Kol'ja",

    "Lavas": "Lavaş",

    "Main Moe -": "Main Moe",
    "Malik N.": "Malik",
    "Mars": "Mars B.",
    "Mighty P.": "Mighty P",
    "Mighty Mo": "Meidi",

    "Nichtdeintyp": "Nicht Dein Typ",

    "O&#39;Spella": "O'Spella",
    
    "P-Zak -": "P-Zak",
    "Pueblo Escobar": "Pablo",
    "Papi Schlauch -": "Papi Schlauch",
    "Phillie": "Phillie.45",
    "Presto -": "Presto",
    "Pretty Shitty Jiggy": "PrettyShittyJiggy",
    "Prettyshittyjiggy": "Pretty Shitty Jiggy",

    "Quasi": "Quasi Infamous",

    "Ryko J": "Ryko-J",

    "Schlawinor": "Schlawin0r",
    "Schlawin0R": "Schlawin0r",
    "Snakeheadrhyme": "SnakeHeadRhyme",

    "Tableddn Timmy": "TableddnTimmy",
    "Tableddntimmy": "TableddnTimmy",
    "Tierstar -": "Tierstar",
    "Tightammic -": "Tightammic",

    "Mr. Whyte": "Yaamann",
    "Yamann": "Yaamann",
    "Mr. White": "Yaamann",
    "Mr White": "Yaamann"
}

df["norm_name #1"] = df["norm_name #1"].replace(aliases)
df["norm_name #2"] = df["norm_name #2"].replace(aliases)

In [129]:
# übrige nan-Werte in namen anzeigen
df[df["name1"].isna() | df["name2"].isna()]

,id,title_raw,upload_date,view_count,like_count,comment_count,url,status,title,info,name1,name2,league,norm_name #1,norm_name #2
215,qtZ8ZEMkfc4,Indoor Stan // Bars vs Herr K. // DLTLLY - Pan...,20200731.0,15465,382,72.0,https://www.youtube.com/watch?v=qtZ8ZEMkfc4,active,Indoor Stan,Bars Vs Herr K. // Dltlly - Panke Garten Berli...,Indoor Stan,None,DLTLLY,Indoor Stan,None
343,c3NhP5ThNps,DLTLLY // Shuffle-T & Marlo VS Henry Bowers & ...,20190106.0,63391,1140,277.0,https://www.youtube.com/watch?v=c3NhP5ThNps,active,DLTLLY,Shuffle-T & Marlo Vs Henry Bowers & Nils M/ Skils,DLTLLY,None,DLTLLY,Dltlly,None
421,RvbRXOjSGY8,DLTLLY // Re-Watching Classics // Bad Bars Bat...,20180517.0,125953,1982,263.0,https://www.youtube.com/watch?v=RvbRXOjSGY8,active,DLTLLY,Re-Watching Classics // Bad Bars Battle (Brian...,DLTLLY,None,DLTLLY,Dltlly,None
543,OxMb1IbK1ao,DLTLLY // RapBattle // Jeder VS Jeden (FREESTYLE),20170504.0,69244,1127,238.0,https://www.youtube.com/watch?v=OxMb1IbK1ao,active,DLTLLY,Rapbattle // Jeder Vs Jeden (Freestyle),DLTLLY,None,DLTLLY,Dltlly,None
550,cQwgmkCYRdQ,DLTLLY // Rap Battles // Teach vs Wandelbarz,20170414.0,18573,336,80.0,https://www.youtube.com/watch?v=cQwgmkCYRdQ,active,DLTLLY,Rap Battles // Teach Vs Wandelbarz,DLTLLY,None,DLTLLY,Dltlly,None
553,kWZkcKo63SY,DLTLLY // Newcomer Battle // Ralle vs Wolff,20170406.0,19977,241,61.0,https://www.youtube.com/watch?v=kWZkcKo63SY,active,DLTLLY,Newcomer Battle // Ralle Vs Wolff,DLTLLY,None,DLTLLY,Dltlly,None
639,arN0RdiC3Xo,DLTLLY // V-Log // Tobi Nice + DuFF vs. Yaramb...,20160702.0,4747,30,12.0,https://www.youtube.com/watch?v=arN0RdiC3Xo,active,DLTLLY,V-Log // Tobi Nice + Duff Vs. Yarambo + Indoor...,DLTLLY,None,DLTLLY,Dltlly,None
705,-F_INtrZZuA,DLTLLY // Re-Watching Classics //Battleboi Bas...,20151201.0,56852,735,101.0,https://www.youtube.com/watch?v=-F_INtrZZuA,active,DLTLLY,Re-Watching Classics //Battleboi Basti & Besse...,DLTLLY,None,DLTLLY,Dltlly,None
711,havRsKXaVec,DLTLLY // Rap Battle // Shazaam vs Brian Damage,20151108.0,30776,392,102.0,https://www.youtube.com/watch?v=havRsKXaVec,active,DLTLLY,Rap Battle // Shazaam Vs Brian Damage,DLTLLY,None,DLTLLY,Dltlly,None
763,cQeD_Je2Xe0,DLTLLY // English Rap Battle // Nils m/ Skils ...,20141227.0,16401,181,33.0,https://www.youtube.com/watch?v=cQeD_Je2Xe0,active,DLTLLY,English Rap Battle // Nils M/ Skils Vs Mos Pro...,DLTLLY,None,DLTLLY,Dltlly,None


In [131]:
# nan-Werte in name - zeilen löschen
df_battles = df.dropna(subset=['name1', 'name2'])

In [133]:
# Battlerapper mit nur 2 Battles (beide Contender) rausfiltern > df_battles_clean

# Alle Namen aus beiden Spalten zählen
all_names = pd.concat([df_battles["norm_name #1"], df_battles["norm_name #2"]])
counts = all_names.value_counts()

# Bedingung: mindestens einer der beiden Namen kommt mehr als einmal vor
mask = df_battles.apply(lambda row: counts[row["norm_name #1"]] > 3 or counts[row["norm_name #2"]] > 3, axis=1)

# Gefiltertes DataFrame
df_battles_clean = df_battles[mask]

In [137]:
df_battles_clean.to_csv("battles_clean.csv")

In [135]:
# Dubletten finden

df_battles_clean["pair_sorted"] = df_battles_clean.apply(
    lambda row: tuple(sorted([row["norm_name #1"], row["norm_name #2"]])),
    axis=1
)

# Schritt 2: alle Paare zählen
pair_counts = df_battles_clean["pair_sorted"].value_counts()

# Schritt 3: nur Paare mit mehr als 1 Auftreten auswählen
duplicate_pairs = pair_counts[pair_counts > 1].index.tolist()

# Schritt 4: Zeilen ausgeben, die zu den Duplikaten gehören
df_duplicates = df_battles_clean[df_battles_clean["pair_sorted"].isin(duplicate_pairs)]

df_duplicates

C:\Users\Rabea\AppData\Local\Temp\ipykernel_3000\2982821227.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_battles_clean["pair_sorted"] = df_battles_clean.apply(


,id,title_raw,upload_date,view_count,like_count,comment_count,url,status,title,info,name1,name2,league,norm_name #1,norm_name #2,pair_sorted
47,GQcwz2t0yJA,Hiding John vs Kid Soldier | Bad Bars Rap Batt...,20240922.0,61842,2767,219.0,https://www.youtube.com/watch?v=GQcwz2t0yJA,active,Hiding John vs. Kid Soldier,Bad Bars Rap Battle @ München | Dltlly,Hiding John,Kid Soldier,DLTLLY,Hiding John,Kid Soldier,"(Hiding John, Kid Soldier)"
62,JCMIDlfCDjY,Proton vs Koozy | On Beat Rap Battle + Intervi...,20240704.0,16771,545,137.0,https://www.youtube.com/watch?v=JCMIDlfCDjY,active,Proton vs. Koozy,On Beat Rap Battle + Interview @ Fob X Dltlly ...,Proton,Koozy,DLTLLY,Proton,Koozy,"(Koozy, Proton)"
210,yHbnx2fuG5c,Steel vs Karanova // DLTLLY RapBattle (Panke /...,20200913.0,20534,542,96.0,https://www.youtube.com/watch?v=yHbnx2fuG5c,active,Steel vs. Karanova,Dltlly Rapbattle (Panke // Berlin) // 2020,Steel,Karanova,DLTLLY,Steel,Karanova,"(Karanova, Steel)"
213,Wdgl_Cg9GKU,Kid Soldier vs Hiding John // DLTLLY RapBattle...,20200809.0,47538,1162,215.0,https://www.youtube.com/watch?v=Wdgl_Cg9GKU,active,Kid Soldier vs. Hiding John,Dltlly Rapbattle (Panke // Berlin) // 2020,Kid Soldier,Hiding John,DLTLLY,Kid Soldier,Hiding John,"(Hiding John, Kid Soldier)"
230,lKlZmLtrRaw,Proton vs Notyzze // DLTLLY OnBeatBattle (NRWe...,20200426.0,173234,3791,415.0,https://www.youtube.com/watch?v=lKlZmLtrRaw,active,Proton vs. Notyzze,Dltlly Onbeatbattle (Nrweekend4 // Duisburg) /...,Proton,Notyzze,DLTLLY,Proton,Notyzze,"(Notyzze, Proton)"
386,iVLMjsx6cuw,Nedal Nib vs CLEP // Studio Version + Extra 4....,20180915.0,111222,2126,217.0,https://www.youtube.com/watch?v=iVLMjsx6cuw,active,Nedal Nib vs. CLEP,Studio Version + Extra 4. Runde // Dltlly Rap ...,Nedal Nib,CLEP,DLTLLY,Nedal Nib,Clep,"(Clep, Nedal Nib)"
391,IYOzBAtlps8,Clep vs Nedal Nib // DLTLLY OnBeatBattle (Spla...,20180826.0,108801,2024,314.0,https://www.youtube.com/watch?v=IYOzBAtlps8,active,Clep vs. Nedal Nib,Dltlly Onbeatbattle (Splash21) // 2018,Clep,Nedal Nib,DLTLLY,Clep,Nedal Nib,"(Clep, Nedal Nib)"
393,34-uHYmnIr8,Yarambo vs Clep // REMATCH! // DLTLLY RapBattl...,20180819.0,226411,3080,594.0,https://www.youtube.com/watch?v=34-uHYmnIr8,active,Yarambo vs. Clep,Rematch! // Dltlly Rapbattle (Hamburg) // 2018,Yarambo,Clep,DLTLLY,Yarambo,Clep,"(Clep, Yarambo)"
526,LSylroi2Toc,Doktor Dave vs. Jack Dragon - REMATCH! // DLTL...,20170716.0,35459,426,122.0,https://www.youtube.com/watch?v=LSylroi2Toc,active,Doktor Dave vs. Jack Dragon - REMATCH!,Dltlly Rapbattle // 2017,Doktor Dave,Jack Dragon - REMATCH!,DLTLLY,Doktor Dave,Jack Dragon,"(Doktor Dave, Jack Dragon)"
644,ZH_LmIdBhAE,Cleptomatic vs Yarambo // DLTLLY RapBattle (MA...,20160612.0,161630,2069,269.0,https://www.youtube.com/watch?v=ZH_LmIdBhAE,active,Cleptomatic vs. Yarambo,Dltlly Rapbattle (Mayhem // Berlin) // 2016,Cleptomatic,Yarambo,DLTLLY,Clep,Yarambo,"(Clep, Yarambo)"


In [63]:
# Liste der rausgeschmissenen nur-2-Battles
df_one_battle = df_battles[~mask]
df_one_battle

,id,title_raw,upload_date,view_count,like_count,comment_count,url,status,title,info,name1,name2,league,norm_name #1,norm_name #2
28,bgoB4s8Mx_c,Duzoe & Ferry vs Sizyphos & Fate | On Beat Rap...,20241106.0,1257,23,1.0,https://www.youtube.com/watch?v=bgoB4s8Mx_c,active,Duzoe & Ferry vs. Sizyphos & Fate,On Beat Rap Battle @ Splash Festival | Dltlly,Duzoe & Ferry,Sizyphos & Fate,DLTLLY,Duzoe & Ferry,Sizyphos & Fate
31,YeUReNWt9ho,LuckyLily & Sh@wdy Sh@f vs. Shizu & Morgana | ...,20241110.0,118223,4390,420.0,https://www.youtube.com/watch?v=YeUReNWt9ho,active,LuckyLily & Sh@wdy Sh@f vs. Shizu & Morgana,On Beat Battle + Interview @ Feel Festival | D...,LuckyLily & Sh@wdy Sh@f,Shizu & Morgana,DLTLLY,Luckylily & Sh@Wdy Sh@F,Shizu & Morgana
33,mmcJqPP9m8c,Cashisclay & 4Tune vs Freezy & Vyrus | Rap Bat...,20241103.0,44957,1409,256.0,https://www.youtube.com/watch?v=mmcJqPP9m8c,active,Cashisclay & 4Tune vs. Freezy & Vyrus,Rap Battle + Interview Splash Festival 2024 | ...,Cashisclay & 4Tune,Freezy & Vyrus,DLTLLY,Cashisclay & 4Tune,Freezy & Vyrus
34,fZyWsFSPtxE,Ferry & Duzoe vs Sizyphos & Fate | 2on2 On Bea...,20241124.0,76278,2295,258.0,https://www.youtube.com/watch?v=fZyWsFSPtxE,active,Ferry & Duzoe vs. Sizyphos & Fate,2On2 On Beat Rap Battle @ Splash| Dltlly,Ferry & Duzoe,Sizyphos & Fate,DLTLLY,Ferry & Duzoe,Sizyphos & Fate
38,LLmwMQb1CQI,Punjizz vs Shizu aka. DeeLah | Rapbattle + Int...,20240912.0,16081,432,83.0,https://www.youtube.com/watch?v=LLmwMQb1CQI,active,Punjizz vs. Shizu aka. DeeLah,Rapbattle + Interview @ München | Dltlly,Punjizz,Shizu aka. DeeLah,DLTLLY,Punjizz,Shizu Aka. Deelah
40,c05Xip_nkxM,Mattis & Sid vs Aladin & Natali | 2on2 Rap Bat...,20241031.0,7509,191,92.0,https://www.youtube.com/watch?v=c05Xip_nkxM,active,Mattis & Sid vs. Aladin & Natali,2On2 Rap Battle @ Proving Grounds | Dltlly,Mattis & Sid,Aladin & Natali,DLTLLY,Mattis & Sid,Aladin & Natali
46,Po2-Ba8ug9I,Onkel Fränk vs Drunk | Rap Battle + Interview ...,20240924.0,8379,162,89.0,https://www.youtube.com/watch?v=Po2-Ba8ug9I,active,Onkel Fränk vs. Drunk,Rap Battle + Interview @ Proving Grounds Wiesb...,Onkel Fränk,Drunk,DLTLLY,Onkel Fränk,Drunk
58,L4bqv0Z187s,Quill vs Hollow Da Don | International Rap Bat...,20240721.0,68567,2508,470.0,https://www.youtube.com/watch?v=L4bqv0Z187s,active,Quill vs. Hollow Da Don,International Rap Battle + Interview @ Mayhem ...,Quill,Hollow Da Don,DLTLLY,Quill,Hollow Da Don
64,q3gQy17LHC0,Mighty P & Lamin vs FFFelix & Nastic | 2on2 Ra...,20240627.0,13486,323,76.0,https://www.youtube.com/watch?v=q3gQy17LHC0,active,Mighty P & Lamin vs. FFFelix & Nastic,2On2 Rap Battle + Interview @ Fob X Dltlly Co-...,Mighty P & Lamin,FFFelix & Nastic,DLTLLY,Mighty P & Lamin,Fffelix & Nastic
67,4j1hDSpgsFw,Kato & Yarambo vs Joseph Steinschleuder & Sizy...,20240616.0,154480,3883,691.0,https://www.youtube.com/watch?v=4j1hDSpgsFw,active,Kato & Yarambo vs. Joseph Steinschleuder & Siz...,2On2 Rap Battle @ Nrweekend | Dltlly,Kato & Yarambo,Joseph Steinschleuder & Sizyphos,DLTLLY,Kato & Yarambo,Joseph Steinschleuder & Sizyphos


In [308]:
# einzelne Spalten individuell anpassen

#df.loc[1037, ["Name #1", "info"]] = ["Ssynic", "BMCL Titelmatch"]
#df.loc[1255, ["Name #1", "info"]] = ["Besser", "BMCL meets DLTLLY Rap Battle"]
#df.loc[688, ["info"]] = ["Merlin abwesend, Craze kickt seine Parts alleine"]
#df.loc[38, ["name2", "info"]] = ["Shizu", "Deelah abwesend, Shizu springt ein | Rapbattle + Interview @ München | Dltlly"]
#df.loc[440, ["norm_name #2", "Location", "info"]] = ["Jollyjay", "Kontext", "Der Sir abwesend, JollyJay springt ein"]
#df.loc[498, ["name1", "name2", "info"]] = ["Ekhead & CPE", "Brian Damage & Karma", "Sport Rap-Battle, DLTLLY & SPOX: Borussia Dortmund vs. FC Bayern München"]
#df.loc[490, ["name1", "name2", "info"]] = ["Bong Teggy", "Lyrico", "Sport Rap-Battle, DLTLLY & SPOX: Fußball vs. Eishockey"]